# Case study: Safe Roads 2022 SAS

## Setup

In [100]:
import numpy as np
import pandas as pd

# Map
from folium import Map
from folium import GeoJson
from folium.plugins import HeatMap

# Plot
import plotly.express as px

## Import data

In [81]:
ksi_df = pd.read_csv('Datos\Canada\ksi.csv')
ksi_df.head()

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,HOUR,STREET1,STREET2,...,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,ObjectId
0,-8.844611e+06,5.412414e+06,3387730.0,892658.0,2006.0,2006/03/11 05:00:00+00,852.0,8.0,BLOOR ST W,DUNDAS ST W,...,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,D11,88.0,High Park North (88),1.0
1,-8.844611e+06,5.412414e+06,3387731.0,892658.0,2006.0,2006/03/11 05:00:00+00,852.0,8.0,BLOOR ST W,DUNDAS ST W,...,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,D11,88.0,High Park North (88),2.0
2,-8.816480e+06,5.434843e+06,3388101.0,892810.0,2006.0,2006/03/11 05:00:00+00,915.0,9.0,MORNINGSIDE AVE,SHEPPARD AVE E,...,<Null>,<Null>,Yes,Yes,<Null>,<Null>,D42,131.0,Rouge (131),3.0
3,-8.816480e+06,5.434843e+06,3388102.0,892810.0,2006.0,2006/03/11 05:00:00+00,915.0,9.0,MORNINGSIDE AVE,SHEPPARD AVE E,...,<Null>,<Null>,Yes,Yes,<Null>,<Null>,D42,131.0,Rouge (131),4.0
4,-8.822759e+06,5.424516e+06,3387793.0,892682.0,2006.0,2006/03/12 05:00:00+00,240.0,2.0,EGLINTON AVE E,COMMONWEALTH AVE,...,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,D41,138.0,Eglinton East (138),5.0


## Cleaning

In [89]:
# Replace Null values with NaN
ksi_df = ksi_df.replace('<Null>', np.nan)

In [90]:
# Data types
ksi_df['INDEX_'] = ksi_df['INDEX_'].astype(int)
ksi_df['ACCNUM'] = ksi_df['ACCNUM'].astype(int)
ksi_df['YEAR'] = ksi_df['YEAR'].astype(int)
ksi_df['DATE'] = pd.to_datetime(ksi_df['DATE']).dt.date
ksi_df['TIME'] = ksi_df['TIME'].astype(int)
ksi_df['HOUR'] = ksi_df['HOUR'].astype(int)
ksi_df['HOOD_ID'] = ksi_df['HOOD_ID'].astype(int)
ksi_df['ObjectId'] = ksi_df['ObjectId'].astype(int)

In [93]:
ksi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 57 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   X                16860 non-null  float64
 1   Y                16860 non-null  float64
 2   INDEX_           16860 non-null  int32  
 3   ACCNUM           16860 non-null  int32  
 4   YEAR             16860 non-null  int32  
 5   DATE             16860 non-null  object 
 6   TIME             16860 non-null  int32  
 7   HOUR             16860 non-null  int32  
 8   STREET1          16860 non-null  object 
 9   STREET2          15350 non-null  object 
 10  OFFSET           2746 non-null   object 
 11  ROAD_CLASS       16363 non-null  object 
 12  DISTRICT         16719 non-null  object 
 13  WARDNUM          16664 non-null  object 
 14  DIVISION         16664 non-null  object 
 15  LATITUDE         16860 non-null  float64
 16  LONGITUDE        16860 non-null  float64
 17  LOCCOORD    

## Initial Analysis

### Ksi locations

In [94]:
# Creación del mapa
for_map = Map(location=[ksi_df['LATITUDE'].mean(), ksi_df['LONGITUDE'].mean()], zoom_start=12, )

In [95]:
# Heatmap
hm_wide = HeatMap(
    list(zip(ksi_df.LATITUDE.values, ksi_df.LONGITUDE.values)),
    min_opacity=0.2,
    radius=17, 
    blur=15, 
    max_zoom=1,
)

In [96]:
for_map.add_child(hm_wide)

### Injuries by age group

In [189]:
# Prepare data
crosstab_age_fatal = pd.crosstab(index=ksi_df['INVAGE'], columns=ksi_df['INJURY'])

In [190]:
# Plot

fig = px.bar(crosstab_age_fatal,
             x=["0 to 4", "5 to 9", "10 to 14", "15 to 19", "20 to 24", "25 to 29", "30 to 34",
                "35 to 39", "40 to 44", "45 to 49", "50 to 54", "55 to 59", "60 to 64", "65 to 69",
                "70 to 74", "75 to 79", "80 to 84", "85 to 89", "90 to 94", "Over 95", "unknown"],
             y=["Fatal", "Major", "Minimal", "Minor", "None"],
             title="Injury based on Age group")
fig.show()

### Injuries by light conditions

In [191]:
# Prepare data
crosstab_light_fatal = pd.crosstab(index=ksi_df['LIGHT'], columns=ksi_df['INJURY'])

In [192]:
# Plot

fig = px.bar(crosstab_light_fatal, title="Injury based on Light condition")
fig.show()

### Injuries by location

In [194]:
# Prepare data
crosstab_loc_fatal = pd.crosstab(index=ksi_df['LOCCOORD'], columns=ksi_df['INJURY'])

In [195]:
# Plot

fig = px.bar(crosstab_loc_fatal, title="Injury based on Light Condition")
fig.show()

### Injuries by visibility condition

In [198]:
# Prepare data
crosstab_visible_fatal = pd.crosstab(index=ksi_df['VISIBILITY'], columns=ksi_df['INJURY'])

In [199]:
# Plot

fig = px.bar(crosstab_visible_fatal, title="Injury based on Light Condition")
fig.show()

### Injuries by incident type

In [205]:
# Prepare data
crosstab_inctype_fatal = pd.crosstab(index=ksi_df['IMPACTYPE'], columns=ksi_df['INJURY'])

In [206]:
# Plot

fig = px.bar(crosstab_inctype_fatal, title="Injury based on Light Condition")
fig.show()

### Accidents by year

In [211]:
# Accidents frequency by year
sin_per_day = ksi_df['YEAR'].value_counts().sort_index()

In [212]:
fig = px.line(sin_per_day)
fig.show()

### Accidents by hour

In [213]:
# Accidents frequency by hour
freq_hour = ksi_df['HOUR'].value_counts().sort_index()

In [218]:
fig = px.bar(freq_hour,
                    labels={
                     "index": "Hour",
                     "value": "Number of Accidents",
                     "variable": "Variable"
                 })

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)

fig.update_layout(title_text='Accidents by hours of the day')

fig.show()

### Accidents by road class

In [221]:
# Accidents frequency by road class
freq_roadclass = ksi_df['ROAD_CLASS'].value_counts().sort_index()

In [225]:
fig = px.bar(freq_roadclass,
                    labels={
                     "index": "Road Class",
                     "value": "Number of Accidents",
                     "variable": "Variable"
                 })

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)

fig.update_layout(title_text='Accidents by hours of the day')

fig.show()

## Selecting Divisions

### Accidents by division

In [227]:
# Accidents frequency by road class
freq_division = ksi_df['DIVISION'].value_counts().sort_index()

In [229]:
fig = px.bar(freq_division,
                    labels={
                     "index": "Division",
                     "value": "Number of Accidents",
                     "variable": "Variable"
                 })

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)

fig.update_layout(title_text='Accidents by Division')

fig.show()